In [ ]:
%%writefile parallel_bfsdfs.cpp
#include <iostream>
#include <vector>
#include <omp.h>
#include <queue>
using namespace std;

class Graph {
    int V;
    vector<vector<int>> adj;

public:
    Graph(int V) : V(V), adj(V) {}

    void addedge(int v, int w) {
        adj[v].push_back(w);
    }

    void paralleldfs(int startvertex) {
        vector<bool> visited(V, false);

        #pragma omp parallel
        {
            #pragma omp single
            {
                paralleldfsutil(startvertex, visited);
            }
        }
    }

    void paralleldfsutil(int v, vector<bool>& visited) {
    bool alreadyVisited;

    // Check and update visited safely
    #pragma omp critical
    {
        alreadyVisited = visited[v];
        if (!alreadyVisited) {
            visited[v] = true;
            cout << v << " ";
        }
    }

    if (alreadyVisited) return;

    for (int n : adj[v]) {
        #pragma omp task firstprivate(n)
        paralleldfsutil(n, visited);
    }

    #pragma omp taskwait
  }
  void parallelBFS(int startVertex) {
        vector<bool> visited(V, false);
        queue<int> q;

        visited[startVertex] = true;
        q.push(startVertex);

        while (!q.empty()) {
            int v = q.front();
            q.pop();
            cout << v << " ";

            #pragma omp parallel for
            for (int i = 0; i < adj[v].size(); ++i) {
                int n = adj[v][i];
                #pragma omp critical
                {
                    if (!visited[n]) {
                        visited[n] = true;
                        q.push(n);
                    }
                }
            }
        }
    }
};

int main() {
    Graph g(7);
    g.addedge(0, 1);
    g.addedge(0, 2);
    g.addedge(1, 3);
    g.addedge(1, 4);
    g.addedge(2, 5);
    g.addedge(2, 6);

    cout << "Parallel DFS output: ";
    g.paralleldfs(0);
    cout << endl;
    cout << "Parallel BFS output: ";
    g.parallelBFS(0);
    return 0;
}

Depth-First Search (DFS): 0 1 3 4 2 5 6 
Breadth-First Search (BFS): 0 1 2 3 4 5 6 



In [ ]:
!g++ -fopenmp parallel_bfsdfs.cpp -o a

In [ ]:
!./a